# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합 - pipeline**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * 단위 테스트
        * 각 기능(함수)에 대해 단계별로 테스트를 수행하며 오류를 해결합니다.
    * 파이프라인 구축
        * 단계1의 결과가 단계2 모델에 input이 되고, 모델의 예측 결과를 기반으로
        * 응급실 추천되도록
        * 조원들이 녹음한 음성 파일에 임의의 좌표(위도, 경도)값을 부여
            * 음성파일 이름과 좌표를 저장하는 별도 데이터셋 생성
        * 각 모듈을 연결하여 파이프라인 구성하는 ipynb 파일 생성



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/미프6차제출/'  #경로 수정 필수

### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [4]:
# 경로 : /content/drive/MyDrive/project6_2/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/미프6차제출/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### 2) 라이브러리 로딩

In [5]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 조에서 생성한 모듈 불러오기 -------------
import emergency

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [6]:
client, model, tokenizer, hospital_info=emergency.load_all(path)#추가로 model, tokenizer, hospital_info도 불러와줌          # api_key 와 map_key 모두 지운상태입니다.(테스트시 본인키 추가)

### (2) audio to text

In [7]:
audiopath=path+'audio1.mp3'#오디오 파일의 경로 지정
text=emergency.a2t(audiopath,client)

지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.



### (3) text summary

In [8]:
summary=emergency.t2s(text,client)#나온 text를 요약

심각한 상황으로 보입니다. 아빠께서 계단에서 넘어지셨고 머리에서 피가 나고 숨을 쉬고 계시지만 어지럽다고 하십니다. 응급실로 즉시 이송이 필요합니다. 계속해서 물을 마시도록 유도해주세요. 지금 119에 전화를 걸고 구체적인 상황을 설명한 뒤 의사 지시에 따라 움직이세요.


### (4) 응급실 등급분류

In [9]:
emergency.predict(summary,model,tokenizer)#지금까지 저장한 summary,model,tokenizer를 넣어 등급  예측

0

### (5) 응급실추천

In [10]:
emergency.recommendation((37.358618,127.115036),hospital_info,os.environ['NAVER_API_ID'],os.environ['NAVER_API_KEY'])#저장한 키와 전화가 온 위치를 기반으로 병원 추천

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,거리
156,분당서울대학교병원,"경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)",권역응급의료센터,031-787-2114,031-787-3119,37.352026,127.124484,2626
116,대진의료재단분당제생병원,경기도 성남시 분당구 서현로180번길 20 (서현동),지역응급의료센터,031-779-0114,031-779-0119,37.387871,127.121328,4866
76,국군수도병원,경기도 성남시 분당구 새마을로177번길 81 (율동),지역응급의료기관,031-725-6075,031-725-6119,37.391867,127.148586,5973


## **3. 파이프라인**

* 세부사항
    * [2. 단계별 테스트] 의 내용을 순차적으로 정리합니다.
        * 데이터 처리 전 준비작업 : 한번 실행하면 되는 영역
            * 키, 데이터로딩
            * 모델/토크나이저 로딩
        * 입력값이 들어 왔을 때 출력값까지 처리되는 영역

In [11]:
def pipeline(path,audio_file,coords):#전체파일경로는 필요하면 입력하고 오디오파일 이름(필요하면 추가 경로),좌표를 입력
  # 0. load key file------------------
  client, model, tokenizer, hospital_info=emergency.load_all(path)
  # 1-1 audio2text--------------------
  audiopath=path+audio_file
  text=emergency.a2t(audiopath,client)
  # 1-2 text2summary------------------
  summary=emergency.t2s(text,client)
  # 2. model prediction------------------
  pred=emergency.predict(summary,model,tokenizer)
  if pred <= 2 :
        print('응급상황 위치기반에 따른 다음과 같은 병원을 추천합니다')
        # 3-1. get_distance------------------
        # 3-2. recommendation----------------
        recommended = emergency.recommendation(
            coords = coords,
            data = hospital_info,
            c_id = os.environ['NAVER_API_ID'],
            c_key = os.environ['NAVER_API_KEY']
            )

        display(recommended)
  else :
        print('비응급')
        print("안정을 취하시고 증상이 계속될 시 의사의 진찰을 받으시길 바랍니다.")


In [14]:
pipeline(path,'audio1.mp3',(37.358618,127.115036))

지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어요. 근데 조금 어지럽다고 하네요. 네네 계단에서 굴렀어요. 지금은 물 마시고 있는데 이거 응급실로 가봐야 할까요? 피도 지금 머졌어요. 네네 나이는 마흔아홉 살 이세요. 어떻게 해야 할지 모르겠어요.

무의식 + 머리 부상 (피가 나옴) + 어지러움
    
응급상황 위치기반에 따른 다음과 같은 병원을 추천합니다


,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,거리
156,분당서울대학교병원,"경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)",권역응급의료센터,031-787-2114,031-787-3119,37.352026,127.124484,2626
116,대진의료재단분당제생병원,경기도 성남시 분당구 서현로180번길 20 (서현동),지역응급의료센터,031-779-0114,031-779-0119,37.387871,127.121328,4225
76,국군수도병원,경기도 성남시 분당구 새마을로177번길 81 (율동),지역응급의료기관,031-725-6075,031-725-6119,37.391867,127.148586,5973


'/content'